<a href="https://colab.research.google.com/github/WangariKimotho/Zindi_Mentorship/blob/main/Mentorship_datacollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##NOTEBOOK ON DATA COLLECTION FROM TWITTER

![Twitter](https://www.itsecurityguru.org/wp-content/uploads/2021/03/alexander-shatov-k1xf2D7jWUs-unsplash.jpg)

This is a task carried out on my first week under the Zindi Mentorship program 2021. The [task](https://zindi.africa/competitions/afd-gender-based-violence-dataset-collection-challenge) is to collect data with information on gender based violence in Africa which has been coined the shadow pandemic affecting women and girls all across Africa.



This is simply a demo on how to go about such a task and with that might not include everything that can be done when collecting data.

####**STEP1:Authorization from twitter**

We need to acquire access from the twitter admin and developer team to collect tweets.This is done first by having a twitter account(if you don't have one) then apply for a developer account to access the API here: https://developer.twitter.com/en/apply-for-access



There are several questions you need to answer before they can authorize your request. It seemed daunting at first to have to answer all those questions, but in retrospect I appreciate that Twitter is taking responsibility and ccountability for the data they have- all these among good AI ethics.


> Once your developer account is setup, create an app that will make use of the API. There will be a set of keys provided to connect to Twitter's API. SPecifically the:


1.   Access token
2.   Access token secret
3.   API key
4.   API secret key


Assign variables to each for use in your code as follows:






In [5]:
##you get these from twitter authorization for eveloper obtained through the link above.
consumer_key = 'wMyunBaUKFTEwjAUPDxTKcwvy'
consumer_secret = 'QwglPQurDyLK3Akc5gL7m0GcWa2TxDZmTm9jAqpfE8gwoNrPiv'
access_token = '3806985075-w30Rn3LMRHviX0Uy9TfLieSwTg0PW07Eq8qjP29'
access_secret = '4AAjV1eUPzOaWOI3jYPTMwjCTCwatATg1tb8kfyOF20er'

####**STEP2:Installing and importing important packages**

In [1]:
!pip install tweepy

In [2]:
!pip install unidecode

     |████████████████████████████████| 241 kB 8.7 MB/s 


In [3]:
#import dependencies
import tweepy
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
from unidecode import unidecode
import time
import datetime
from tqdm import tqdm 
import pandas as pd  
import numpy as np 

####Connect to twitter API using the secrets

In [6]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)
###wait on rate when set to TRUE allows to continue with search on scipt onec the limit is reached, if FALSE, the script stops running


We then define a function that will take our searh query.

In [ ]:
def tweetSearch(query, limit):
    """
    This function will search a query provided in the twitter and,
    retun a list of all tweets that have a query. 
    """

    # Create a blank list variable on which to append the tweets
    tweets = []
    
    # Iterate through Twitter using Tweepy to find our query with our defined limit
    for page in tweepy.Cursor(
        api.search, q=query, count=limit, tweet_mode="extended"
    ).pages(limit):
        for tweet in page:
            tweets.append(tweet)

    # return tweets
    return tweets

We create another function to save the tweets obtained in a dataframe

In [ ]:
def tweets_to_data_frame(tweets):
    """
    This function will receive tweets and collect specific data from it such as place, tweet's text,likes 
    retweets and save them into a pandas data frame.
    
    This function will return a pandas data frame that contains data from twitter.
    """
    df = pd.DataFrame(data=[tweet.full_text.encode('utf-8') for tweet in tweets], columns=["Tweets"])

    df["length"] = np.array([len(tweet.full_text) for tweet in tweets])
    df["date"] = np.array([tweet.created_at for tweet in tweets]) #tweepy only takes tweets within the last 7 days
    df["coordinateS"] = np.array([tweet.coordinates for tweet in tweets])
    df['location'] = np.array([tweet.user.location for tweet in tweets]) #often obtained from the user's twitter bio, remember we need tweets mainly based on Africa
    df["Language"] = np.array([tweet.lang for tweet in tweets])
    df["source"] = np.array([tweet.source for tweet in tweets])
    df["likes"] = np.array([tweet.favorite_count for tweet in tweets])
    df["retweets"] = np.array([tweet.retweet_count for tweet in tweets])

    return df

###Adding twitter hashtags based on Gender Based Violence

Some of those listed below are based on a country's case that spiralled twitter posts. Most were acquired from doing some research on the countries' twitter presence especially with regard to Anti-GBV advocacy.

In [ ]:
hashtags = [
'#GBV',#general
'#sexism', #general
'#rape', #general
'#domesticviolence', #general
'#mydressmychoice', #Kenyan based
'#ShutItAllDownNamibia', #Namibian based
'#TheTotalShutdown', #SA based
'#TotalshutdownKE', #Kenyan based
'#sayhername', #global
'#shutitdown', #SA based
'#JusticeForNoura', #Sudan based
'#JusticeForOchanya', #Nigerian based
'#BringBackOurGirls', #Nigerian based
'#metoo'  #global
]

##you can add more hashtags

We run both functions and collect data based on the above hashtags.The following code may take a while to complete so be patient:)

In [ ]:
total_tweets = 0

"""
The following for loop will collect a tweets that have the hashtags
 mentioned in the list and save the tweets into csv file
"""

for n in tqdm(hashtags):
    # first we fetch all tweets that have specific hashtag
    hash_tweets = tweetSearch(query=n,limit=7000)
    total_tweets += int(len(hash_tweets))
    
    # second we convert our tweets into datarame
    df = tweets_to_data_frame(hash_tweets)
    
    #third we save the dataframe into csv file
    df.to_csv("{}_tweets.csv".format(n))

In [ ]:
# count total number of tweets collected
print("total_tweets: {}".format(total_tweets))

In [ ]:
df1 = pd.read_csv('#BringBackOurGirls_tweets.csv')
df1

In [ ]:
(df1['location']).value_counts() ###look into the main locations from our dataset

In [ ]:
df1[df1['location']=='Nigeria']

In [ ]:
df2 = pd.read_csv('#TotalshutdownKE_tweets.csv')
df2

In [ ]:
(df2['location']).value_counts()

In [ ]:
df3 =pd.read_csv('#GBV_tweets.csv')
df3

In [ ]:
(df3[df3['location']==str('South Africa')])

Cleaning up the tweets, here is  sneak peek on how to do that. Note that this notebook was mainly centred around collecting data from tweets.

In [ ]:
import re# function to remove special characters
def remove_special_characters(text):
    # define the pattern to keep
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    return re.sub(pat, '', text)
 
# call function
remove_special_characters(“007 Not sure@ if this % was #fun! 558923 What do# you think** of it.? $500USD!”)

In [ ]:
# remove twitter handles (@user)

## remove special characters, numbers, punctuations
df3['clean_tweet'] = df3['Tweets'].str.replace("[^a-zA-Z#]", " ")
#      
df3['clean_tweet'] = df3['Tweets'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2])) 
  
df3['clean_tweet']

0       b'"To #EndFGM and #GenderBasedViolence young p...
1       b'RT @glocalreporting: Women suffer even more ...
2       b"RT @CedawPT: '40% women have experienced som...
3       b'RT @EUinCV: fight gender based violence impo...
4       b'To fight gender based violence important hav...
                              ...                        
2123    b'RT @JulittaOnabanjo: IASC PSEAH champion, US...
2124    b'RT @JulittaOnabanjo: IASC PSEAH champion, US...
2125    b'RT @salhaj: @Atayeshe @UNFPA \nFrom medical ...
2126    b'RT @salhaj: @Atayeshe @UNFPA \nFrom medical ...
2127    b"RT @NCADV: The proposed amendments bar intim...
Name: clean_tweet, Length: 2128, dtype: object

####Merging the datasets into one

In [ ]:
df_all = pd.concat([df1,df2,df3])
df_all

,Unnamed: 0,Tweets,length,date,coordinateS,location,Language,source,likes,retweets,clean_tweet
0,0,b'RT @its_kachi: Id rather BokoHaram take my d...,140,2021-05-24 13:54:27,NaN,NaN,en,Twitter for iPhone,0,6,NaN
1,1,"b""RT @chibok276: We've been missing for 2596 d...",67,2021-05-24 13:45:07,NaN,NaN,en,Twitter for iPhone,0,1,NaN
2,2,b'RT @its_kachi: Id rather BokoHaram take my d...,140,2021-05-24 13:12:27,NaN,"Doing bear stuff Capitola, FL",en,Twitter Web App,0,6,NaN
3,3,b'RT @DolapoAina: #BringBackOurGirls DAY2598\n...,140,2021-05-24 11:31:37,NaN,"Lagos, Nigeria",und,Twitter Web App,0,2,NaN
4,4,b'#BringBackOurGirls \n#HopeEndures \n#BBOG ht...,63,2021-05-24 10:27:31,NaN,NaN,und,Twitter Web App,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2123,2123,"b'RT @JulittaOnabanjo: As IASC PSEAH champion,...",140,2021-05-16 22:26:07,NaN,Sweden,en,Twitter Web App,0,20,"b'RT @JulittaOnabanjo: IASC PSEAH champion, US..."
2124,2124,"b'RT @JulittaOnabanjo: As IASC PSEAH champion,...",140,2021-05-16 22:23:46,NaN,Instagram @atayesheunfpa,en,Twitter Web App,0,20,"b'RT @JulittaOnabanjo: IASC PSEAH champion, US..."
2125,2125,b'RT @salhaj: Dr @Atayeshe @UNFPA \nFrom medic...,144,2021-05-16 22:22:48,NaN,Instagram @atayesheunfpa,en,Twitter Web App,0,11,b'RT @salhaj: @Atayeshe @UNFPA \nFrom medical ...
2126,2126,b'RT @salhaj: Dr @Atayeshe @UNFPA \nFrom medic...,144,2021-05-16 22:10:16,NaN,Senegal,en,Twitter for iPhone,0,11,b'RT @salhaj: @Atayeshe @UNFPA \nFrom medical ...


Below is another method of collecting live tweets from twitter by specifying the granularity. In my case I chose the country Kenya.

In [8]:
!pip install twitter

     |████████████████████████████████| 50 kB 3.7 MB/s 


In [10]:
from twitter import *
t = Twitter(auth=OAuth(access_token, access_secret,consumer_key,consumer_secret))
result = t.geo.search(query="Kenya", granularity="country")
place_id = result['result']['places'][0]['id']

result = t.search.tweets(q="place:%s" % place_id)
for tweet in result['statuses']:
    print (tweet['text'] + " ~~ " + tweet['place']['name'] if tweet['place'] else "Undefined place")

From the streets she rose, to the streets she returned.
Dennis Okari dodged a bullet a stunt only seen in movies.
M… https://t.co/L107lEzzEn ~~ Kabete
@Sphinx_jf Labda umevaa lingerie inakuwinkia ~~ Mombasa
@muhammadljunaid Hello am a photographer from nairobi kenya
My instagram handle is @simon__kavita https://t.co/mKALaE6p2l ~~ Nairobi
@alfu_254 Alifungua prisoners wote juzi 😂😂 ~~ Nairobi
@GSaruni @veeeyecee Ssd ~~ Mombasa
@Ntobi_ @DKAmbTanzania Viongozi wajihoji na wajiamgalie,Daah Pole kamanda ~~ Mombasa
🤣🤣🤣🤣🤣🤣 https://t.co/35F7QyR1cE ~~ Nairobi
Ta gúi https://t.co/3TwWN7zVyz ~~ Eldoret
Chairman @kipmurkomen , what a low! https://t.co/oUzkcFy7lo ~~ Nairobi
@BrunoSigwela This is typically the best art I have come across ~~ Kabete
CCTV footage has emerged showing the moment a boda boda rider grabbed a mobile phone from a {police officer} and sp… https://t.co/FOOHe51ocH ~~ Nairobi
✦ [ Wash Wash ]

Hilarious CCTV Video: Boda Boda Rider Robbing Police Officer

https://t.co/97eSuMkpvO ~~

Some more resources that show how to acquire data from twitter can be found [here](https://www.earthdatascience.org/courses/use-data-open-source-python/intro-to-apis/twitter-data-in-python/) or [here](https://towardsdatascience.com/an-extensive-guide-to-collecting-tweets-from-twitter-api-v2-for-academic-research-using-python-3-518fcb71df2a) or on the [tweepy documentation page](https://docs.tweepy.org/en/latest/)

We can ofcourse turn this into a text cleaning task or even find a form of machine learning classification task based on the data collected. This challenge was however mainly based on collecting data.